In [5]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("HUMETRO_LOCAL_RAG_EVAL_TEST")

LangSmith 추적을 시작합니다.
[프로젝트명]
HUMETRO_LOCAL_RAG_EVAL_TEST


In [29]:
import subprocess
from typing import Any
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma


# 2. 임베딩 모델 생성
def create_embeddings(model_name: str = "text-embedding-3-small") -> OpenAIEmbeddings:
    return OpenAIEmbeddings(model=model_name, dimensions=1536)


def load_vectorstore(persist_directory: str, embeddings: Any) -> Chroma:
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
        collection_name="rag_documents",
    )
    print(f"벡터스토어 로드 완료. 문서 수: {vectorstore._collection.count()}")
    return vectorstore


# 3. 검색기(retriever) 생성 함수
def create_retriever(vectorstore: Chroma, k: int = 4) -> Any:
    return vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": k})


# 4. LLM 모델 생성 함수
def create_llm(model_name: str, temperature: float = 0.1) -> Any:
    if "gpt" not in model_name:
        # Ollama 모델 확인 및 없으면 다운로드
        try:
            result = subprocess.run(
                ["ollama", "ps"], capture_output=True, text=True, check=True
            )
            running_servers = result.stdout.strip()

            # 2. 실행 중인 모델이 있으면 종료
            if running_servers:
                print("현재 실행 중인 모델을 종료합니다...")
                for line in running_servers.split("\n"):
                    if line and not line.startswith("NAME"):  # 헤더 행 제외
                        running_model = line.split()[0]
                        # 실행 중인 모델이 요청된 모델과 다른 경우에만 종료
                        if running_model != model_name:
                            print(f"모델 {running_model}을 종료합니다...")
                            subprocess.run(
                                ["ollama", "stop", running_model],
                                check=True,
                                capture_output=True,
                            )
            # 사용 가능한 모델 목록 확인
            result = subprocess.run(
                ["ollama", "list"], capture_output=True, text=True, check=True
            )
            available_models = result.stdout.lower()

            # 모델 이름에서 태그 분리
            if ":" in model_name:
                base_model = model_name.split(":")[0]
            else:
                base_model = model_name

            # 모델이 없으면 다운로드
            if base_model not in available_models:
                print(f"모델 {model_name}을 다운로드합니다...")
                subprocess.run(["ollama", "pull", model_name], check=True)
                print(f"모델 {model_name} 다운로드 완료")
            else:
                print(f"모델 {model_name}이 이미 존재합니다.")

        except subprocess.CalledProcessError as e:
            print(f"Ollama 명령 실행 중 오류 발생: {e}")
            print("Ollama가 설치되어 있고 실행 중인지 확인하세요.")
        except Exception as e:
            print(f"Ollama 모델 준비 중 오류 발생: {e}")

        # Ollama 모델 연결
        return Ollama(model=model_name, temperature=temperature)
    else:
        # OpenAI 모델 사용
        return ChatOpenAI(model=model_name, temperature=temperature)


# 5. RAG 체인 생성 함수
def create_rag_chain(llm: Any, retriever: Any) -> Any:
    # 한국의 역무환경을 고려한 RAG 프롬프트 템플릿
    template = """
당신은 한국의 도시철도 역무 지식 도우미입니다.
주어진 질문에 대해 제공된 문맥 정보를 기반으로 정확하고 도움이 되는 답변을 제공하세요.
문맥에 없는 내용은 답변하지 마세요. 모르는 경우 솔직히 모른다고 말하세요.

문맥 정보:
{context}

질문: {question}

답변:
"""

    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join([doc.page_content for doc in docs])

    # LCEL을 사용한 RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [30]:
import json
import re
from tqdm import tqdm
from langchain_openai import OpenAIEmbeddings

embeddings = create_embeddings()
vectorstore = load_vectorstore(persist_directory="vectorstore", embeddings=embeddings)
if len(vectorstore.similarity_search("서울역 주변 명소")) == 0:
    raise ValueError("vectorstore is empty")

벡터스토어 로드 완료. 문서 수: 150


In [31]:
models = {
    "exaone": "exaone3.5",
    "clova": "yoonyoung/HyperCLOVAX-SEED-Vision-Instruct-3B",
    "kanana": "huihui_ai/kanana-nano-abliterated",  # 2.1b
    "llama": "benedict/linkbricks-llama3.1-korean:8b",  # 8b quantized
    "gpt-4o-mini": "gpt-4o-mini",
}


def sanitize_filename(filename):
    return re.sub(r"[^a-zA-Z0-9_.]", "_", filename)[:20] + ".json"


for model_name, signature in models.items():
    print(f"Loading {model_name}...")
    llm = create_llm(signature)
    retriever = create_retriever(vectorstore)
    rag_chain = create_rag_chain(llm, retriever)
    result_list = []
    question_list = [
        "지하철 역에 대해 알려줘",
        "서면역의 주변 명소를 알려줘",
        "어른1 아이 2명의 1구간 요금은?",
    ]
    for question in tqdm(question_list, desc=f"Evaluating {model_name}"):
        result = rag_chain.invoke(question)
        result_list.append({"question": question, "answer": result})
    with open(sanitize_filename(f"result_{model_name}.json"), "w") as f:
        json.dump(result_list, f, ensure_ascii=False)
    print()


Loading exaone...
현재 실행 중인 모델을 종료합니다...
모델 exaone3.5이 이미 존재합니다.


Evaluating exaone: 100%|██████████| 3/3 [00:59<00:00, 19.89s/it]
pulling manifest ⠙ 


Loading clova...
현재 실행 중인 모델을 종료합니다...
모델 exaone3.5:latest을 종료합니다...
모델 yoonyoung/HyperCLOVAX-SEED-Vision-Instruct-3B을 다운로드합니다...


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 63fcaa2d012b: 100% ▕██████████████████▏   29 B                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 32ba8b50b4ce: 100% ▕██████████████████▏  635 B                         
verifying sha256 digest 
writing manifest 
success 


모델 yoonyoung/HyperCLOVAX-SEED-Vision-Instruct-3B 다운로드 완료


...

Evaluating kanana: 100%|██████████| 3/3 [00:25<00:00,  8.41s/it]



Loading llama...
현재 실행 중인 모델을 종료합니다...
모델 huihui_ai/kanana-nano-abliterated:latest을 종료합니다...
모델 benedict/linkbricks-llama3.1-korean:8b이 이미 존재합니다.


Evaluating llama: 100%|██████████| 3/3 [01:00<00:00, 20.21s/it]



Loading gpt-4o-mini...


Evaluating gpt-4o-mini: 100%|██████████| 3/3 [00:08<00:00,  2.93s/it]